In [ ]:
# Personal Finance Chatbot - Google Colab Setup
# Run this entire code in ONE cell by pressing Shift+Enter

# ============================================
# CELL 1: Install Required Libraries
# ============================================
print("📦 Installing required libraries...")
print("This may take 2-3 minutes...\n")

!pip install -q transformers torch accelerate
!pip install -q gradio pandas numpy

print("✅ Installation complete!")

# ============================================
# CELL 2: Import Libraries
# ============================================
print("📚 Importing libraries...")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import pandas as pd
import json
from datetime import datetime

print("✅ Libraries imported successfully!")

# ============================================
# CELL 3: Initialize IBM Granite Model
# ============================================
print("🤖 Loading IBM Granite 3.3 2B Instruct model...")
print("This will take 3-5 minutes (downloading ~4.5GB model)...\n")

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "ibm-granite/granite-3.3-2b-instruct",
    torch_dtype=torch.float16,
    device_map="auto"
)

print("✅ Model loaded successfully!")
print(f"Device: {model.device}")

# ============================================
# CELL 4: Define Chatbot Class
# ============================================
print("🏗️ Setting up chatbot class...")

class FinanceChatbot:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.conversation_history = []
        self.user_profile = {}

    def detect_user_type(self, user_input):
        """Detect if user is a student or professional"""
        student_keywords = ['student', 'college', 'university', 'study', 'course']
        professional_keywords = ['job', 'work', 'salary', 'professional', 'career']

        input_lower = user_input.lower()
        if any(keyword in input_lower for keyword in student_keywords):
            return 'student'
        elif any(keyword in input_lower for keyword in professional_keywords):
            return 'professional'
        return 'general'

    def build_system_prompt(self, user_type='general'):
        """Build context-aware system prompt"""
        base_prompt = """You are a helpful Personal Finance Assistant powered by IBM Granite AI.
Provide practical financial guidance on savings, taxes, investments, budgeting, and spending."""

        if user_type == 'student':
            return base_prompt + """
Focus on student budgeting, part-time income management, and building financial habits.
Use simple, friendly language with relatable examples."""
        elif user_type == 'professional':
            return base_prompt + """
Focus on tax optimization, investment portfolios, retirement planning, and advanced strategies.
Use professional tone with detailed, data-driven insights."""
        return base_prompt + "\nProvide clear, actionable advice for all audiences."

    def generate_response(self, user_message, user_type='general', max_tokens=400):
        """Generate AI response"""
        system_prompt = self.build_system_prompt(user_type)

        messages = [{"role": "system", "content": system_prompt}]

        # Add recent conversation history
        for msg in self.conversation_history[-6:]:
            messages.append(msg)

        messages.append({"role": "user", "content": user_message})

        # Tokenize
        inputs = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(self.model.device)

        # Generate
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )

        # Update history
        self.conversation_history.append({"role": "user", "content": user_message})
        self.conversation_history.append({"role": "assistant", "content": response})

        return response

    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        return "✅ Conversation history cleared!"

# Initialize chatbot
chatbot = FinanceChatbot(tokenizer, model)

print("✅ Chatbot initialized!")

# ============================================
# CELL 5: Create Gradio Interface
# ============================================
print("🎨 Creating Gradio interface...")

def chat_interface(message, history, user_type):
    """Gradio chat interface function"""
    if not message.strip():
        return history, history

    # Generate response
    response = chatbot.generate_response(message, user_type.lower())

    # Update history
    history.append([message, response])

    return history, history

def clear_chat():
    """Clear chat history"""
    chatbot.clear_history()
    return [], []

def quick_budget_analysis(income, rent, food, transport, utilities, other):
    """Quick budget analysis"""
    total_expenses = rent + food + transport + utilities + other
    savings = income - total_expenses
    savings_rate = (savings / income * 100) if income > 0 else 0

    expenses = {
        "Rent": rent,
        "Food": food,
        "Transportation": transport,
        "Utilities": utilities,
        "Other": other
    }

    # Generate AI analysis
    prompt = f"""Analyze this budget:
Income: ₹{income:,.2f}
Expenses: {json.dumps(expenses)}
Total Expenses: ₹{total_expenses:,.2f}
Savings: ₹{savings:,.2f}
Savings Rate: {savings_rate:.1f}%

Provide brief analysis and 3 specific recommendations."""

    analysis = chatbot.generate_response(prompt, 'general', max_tokens=300)

    return f"""
📊 BUDGET SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━
💰 Monthly Income: ₹{income:,.2f}
💸 Total Expenses: ₹{total_expenses:,.2f}
💵 Net Savings: ₹{savings:,.2f}
📈 Savings Rate: {savings_rate:.1f}%

{f"🎯 Status: {'Excellent!' if savings_rate >= 30 else 'Good' if savings_rate >= 20 else 'Needs Improvement'}" if savings_rate > 0 else '⚠️ Warning: Expenses exceed income!'}

📋 EXPENSE BREAKDOWN:
- Rent: ₹{rent:,.2f} ({rent/total_expenses*100:.1f}%)
- Food: ₹{food:,.2f} ({food/total_expenses*100:.1f}%)
- Transport: ₹{transport:,.2f} ({transport/total_expenses*100:.1f}%)
- Utilities: ₹{utilities:,.2f} ({utilities/total_expenses*100:.1f}%)
- Other: ₹{other:,.2f} ({other/total_expenses*100:.1f}%)

━━━━━━━━━━━━━━━━━━━━━━━
🤖 AI ANALYSIS:
{analysis}
"""

def get_investment_advice(age, amount, risk_level, goal):
    """Get investment advice"""
    prompt = f"""Provide investment advice:
Age: {age}
Investment Amount: ₹{amount:,.2f}
Risk Tolerance: {risk_level}
Goal: {goal}

Give specific investment recommendations with percentages."""

    return chatbot.generate_response(prompt, 'professional', max_tokens=400)

# Create Gradio Blocks interface
with gr.Blocks(theme=gr.themes.Soft(), title="Personal Finance Chatbot") as demo:

    gr.Markdown("""
    # 💰 Personal Finance Chatbot
    ### Powered by IBM Granite 3.3 2B Instruct AI
    Get personalized financial advice, budget analysis, and investment guidance!
    """)

    with gr.Tab("💬 Chat Assistant"):
        gr.Markdown("Ask me anything about personal finance, budgeting, savings, investments, or taxes!")

        user_type_radio = gr.Radio(
            choices=["General", "Student", "Professional"],
            value="General",
            label="I am a:",
            info="Select your profile for personalized advice"
        )

        chatbot_interface = gr.Chatbot(
            label="Conversation",
            height=400,
            show_label=True
        )

        msg = gr.Textbox(
            label="Your Message",
            placeholder="Type your financial question here...",
            lines=2
        )

        with gr.Row():
            submit_btn = gr.Button("Send 📨", variant="primary")
            clear_btn = gr.Button("Clear History 🗑️")

        # Quick action buttons
        gr.Markdown("### Quick Actions:")
        with gr.Row():
            savings_btn = gr.Button("💵 Savings Tips")
            invest_btn = gr.Button("📈 Investment Guide")
            tax_btn = gr.Button("🧾 Tax Planning")
            goal_btn = gr.Button("🎯 Set Goal")

        # Event handlers for quick actions
        savings_btn.click(
            lambda: "What are some effective ways to save money each month?",
            outputs=msg
        )
        invest_btn.click(
            lambda: "I want to start investing. Where should I begin?",
            outputs=msg
        )
        tax_btn.click(
            lambda: "Can you explain tax-saving strategies in India?",
            outputs=msg
        )
        goal_btn.click(
            lambda: "Help me set a financial goal for building an emergency fund",
            outputs=msg
        )

        # Event handlers
        submit_btn.click(
            chat_interface,
            inputs=[msg, chatbot_interface, user_type_radio],
            outputs=[chatbot_interface, chatbot_interface]
        ).then(
            lambda: "",
            outputs=msg
        )

        msg.submit(
            chat_interface,
            inputs=[msg, chatbot_interface, user_type_radio],
            outputs=[chatbot_interface, chatbot_interface]
        ).then(
            lambda: "",
            outputs=msg
        )

        clear_btn.click(clear_chat, outputs=[chatbot_interface, chatbot_interface])

    with gr.Tab("📊 Budget Analyzer"):
        gr.Markdown("### Enter your monthly income and expenses for instant analysis")

        with gr.Row():
            income_input = gr.Number(label="💰 Monthly Income (₹)", value=50000)

        gr.Markdown("#### Monthly Expenses:")
        with gr.Row():
            rent_input = gr.Number(label="🏠 Rent/Housing", value=15000)
            food_input = gr.Number(label="🍕 Food", value=8000)

        with gr.Row():
            transport_input = gr.Number(label="🚗 Transportation", value=3000)
            utilities_input = gr.Number(label="💡 Utilities", value=2000)

        with gr.Row():
            other_input = gr.Number(label="🛍️ Other Expenses", value=5000)

        analyze_btn = gr.Button("Analyze Budget 📊", variant="primary", size="lg")

        budget_output = gr.Textbox(
            label="Budget Analysis & Recommendations",
            lines=20,
            show_label=True
        )

        analyze_btn.click(
            quick_budget_analysis,
            inputs=[income_input, rent_input, food_input, transport_input,
                   utilities_input, other_input],
            outputs=budget_output
        )

    with gr.Tab("📈 Investment Advisor"):
        gr.Markdown("### Get personalized investment recommendations")

        with gr.Row():
            age_input = gr.Slider(18, 65, value=28, label="Age", step=1)
            amount_input = gr.Number(label="Investment Amount (₹)", value=50000)

        with gr.Row():
            risk_input = gr.Radio(
                choices=["Low", "Moderate", "High"],
                value="Moderate",
                label="Risk Tolerance"
            )
            goal_input = gr.Textbox(
                label="Investment Goal",
                value="Build emergency fund",
                placeholder="e.g., Retirement, Buy house, Emergency fund"
            )

        invest_btn = gr.Button("Get Investment Advice 📈", variant="primary")

        invest_output = gr.Textbox(
            label="Investment Recommendations",
            lines=15,
            show_label=True
        )

        invest_btn.click(
            get_investment_advice,
            inputs=[age_input, amount_input, risk_input, goal_input],
            outputs=invest_output
        )

    with gr.Tab("ℹ️ About"):
        gr.Markdown("""
        ## About Personal Finance Chatbot

        This chatbot uses **IBM Granite 3.3 2B Instruct**, an advanced AI model designed
        to provide intelligent financial guidance.

        ### Features:
        - 💬 **Natural Conversations** - Ask any finance question naturally
        - 👥 **Profile-Based Advice** - Customized for students and professionals
        - 📊 **Budget Analysis** - Comprehensive expense tracking and insights
        - 📈 **Investment Guidance** - Personalized investment recommendations
        - 🧾 **Tax Planning** - Tax-saving strategies and tips
        - 🎯 **Goal Setting** - Help with financial goal planning

        ### How to Use:
        1. **Chat Tab**: Have natural conversations about any financial topic
        2. **Budget Analyzer**: Enter your income and expenses for instant analysis
        3. **Investment Advisor**: Get personalized investment recommendations

        ### Model Information:
        - **Model**: IBM Granite 3.3 2B Instruct
        - **Parameters**: 2 Billion
        - **Capabilities**: Financial advice, budgeting, investment guidance

        ### Tips for Best Results:
        - Be specific about your financial situation
        - Mention your age, income range for better advice
        - Ask follow-up questions for clarity
        - Use the profile selector for personalized responses

        ---
        **Note**: This is an AI assistant for educational purposes. Always consult
        with certified financial advisors for major financial decisions.
        """)

print("✅ Gradio interface created!")

# ============================================
# CELL 6: Launch the Application
# ============================================
print("\n" + "="*60)
print("🚀 LAUNCHING PERSONAL FINANCE CHATBOT")
print("="*60 + "\n")

# Launch with public link
demo.launch(
    share=True,  # Creates public link
    debug=True,
    show_error=True
)

print("\n✅ Application is running!")
print("📱 Click the public URL above to access from any device")
print("⏹️  Press Ctrl+C to stop the server")

📦 Installing required libraries...
This may take 2-3 minutes...

✅ Installation complete!
📚 Importing libraries...
✅ Libraries imported successfully!
🤖 Loading IBM Granite 3.3 2B Instruct model...
This will take 3-5 minutes (downloading ~4.5GB model)...



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully!
Device: cuda:0
🏗️ Setting up chatbot class...
✅ Chatbot initialized!
🎨 Creating Gradio interface...


/tmp/ipython-input-1841392705.py:240: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(


✅ Gradio interface created!

🚀 LAUNCHING PERSONAL FINANCE CHATBOT

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e5ea7cf5345401c3f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
